# Introduction to lifpy

In [22]:
import lifpy as lfp

# 1. Make your topography input datasets.  
First thing you need to do is to create input topography dataset to run LISFLOOD-FP.  
lifpy.PreProcess is a module to create those input files in a required format.  

## 1.1 Required input data source  
First and foremost, you need to prepare the input files for the simulation. The current version of lifpy needs the following datasets in a specific format: 
- __Hydrography data (GeoTiff)__
  * Flow direction
  * River width
  * Upstream area size
  * Surface elevation
- __River discharge data (netCDF4)__
  * shoud contain time and id (river identification number or string) coordinates.

## 1.2 Generate input topography data
Once your data is ready:
```python
import lifpy as lfp

# define required arguments
elvPath = "list of path to the surface elevation files"
upaPath = "list of path to the surface upstream area size files"
wthPath = "list of path to the surface river width files"
thsld = "the minimum size of upstrea area. The pixel whose upsream area size is above this number will be considered as rivers"
nCols = "number of files for a longitudinal axis"
nRaws = "number of files for a latitudinal axis"

lfp.PreProcess.mfpreprocess(upaPath, elvPath, wthPath, thsld, nCols, nRaws)
```
will create a topography data required for your lisflood-fp simulation. In this example, we have the files in example/hydrography directory.  

In [2]:
elvPath = ["./examples/hydrography/n35w100_elv.tif","./examples/hydrography/n35w095_elv.tif"]
upaPath = ["./examples/hydrography/n35w100_upa.tif","./examples/hydrography/n35w095_upa.tif"]
wthPath = ["./examples/hydrography/n35w100_wth.tif","./examples/hydrography/n35w095_wth.tif"]
thsld = 24.04
nCols = 2
nRaws = 1

Here, we have two input GeoTiff sources. Usually those files are equally splitted into tiles.  
lifpy.PreProcess.mfpreprocess() will process those multiple files into a single formatted text file LISFLOOD-FP can read.  
  
Those files are specified in elvPath (elevation), upaPath (upstream area size), wthPath (river width) and must be aligned in a C-order.  
nCols and nRaws is a number of tiles in a longitudinal and latitudinal axis, respectively.  

In [3]:
lfp.PreProcess.mfpreprocess(upaPath, elvPath, wthPath, thsld, nCols, nRaws, prefix="example")

./examples/hydrography/n35w100_elv.tif
./examples/hydrography/n35w095_elv.tif
output file informaton:
ncols 12000
nrows 6000
xllconer -100.000000
yllcorner 35.000000
cellsize 0.000833
NODATA_value -9999

./out/example.dem.ascii
./examples/hydrography/n35w100_upa.tif
./examples/hydrography/n35w095_upa.tif
./examples/hydrography/n35w100_wth.tif
./examples/hydrography/n35w095_wth.tif
./out/example.width.asc
./out/example.bank.asc


This will create cache/ and out/ directory.  
The generated files are stored in out/ directory.  

In [5]:
!ls ./out

example.bank.asc  example.dem.ascii  example.width.asc


This is the file looks like:

In [8]:
!head -n7 ./out/example.dem.ascii

ncols 12000
nrows 6000
xllconer -100.000000
yllcorner 35.000000
cellsize 0.000833
NODATA_value -9999
689.60004,690.3,689.60004,688.60004,686.7,684.8,687.7,688.8,689.7,689.2,690.0,691.8,692.9,694.8,698.9,696.9,691.60004,689.7,690.5,695.2,699.9,702.4,704.9,709.9,715.7,718.60004,718.7,721.60004,722.5,722.3,719.2,716.3,711.7,711.4,712.7,710.0,703.9,698.4,698.2,701.8,697.9,696.0,702.7,708.3,713.60004,717.60004,718.10004,718.5,722.3,723.4,721.60004,719.7,715.8,710.9,704.0,703.0,703.9,706.7,713.60004,718.5,722.5,724.4,723.4,720.4,714.7,707.9,704.0,704.0,701.10004,698.10004,697.9,694.0,694.0,695.0,700.4,703.2,701.5,699.5,702.0,702.0,702.9,704.0,706.10004,708.0,711.9,713.7,716.60004,718.4,716.60004,718.5,724.10004,725.2,725.2,726.10004,727.10004,729.10004,729.4,726.4,720.7,715.8,714.7,712.60004,708.7,711.7,717.4,721.8,725.60004,726.0,726.4,727.2,725.5,723.60004,720.7,716.9,715.9,715.0,713.8,712.60004,708.60004,706.60004,706.60004,707.4,707.3,706.3,705.4,702.7,700.9,701.7,700.8,699.9,697.9,698.7

# 2. Make your input forcing datasets
The next thing to do is creating input discharge hydrographs and locate them onto a LISFLOOD-FP model grid coordinates.  
lifpy uses upstream area information of both sides (input discharge and model grid coordinates) and compares them to adjust lat/lon location. 

The current version of lifpy needs the following datasets in a specific format:
- __Discharge data (netCDF)__
  * discharge time series at each point (2d array)
  * time index (dimension 1)
  * river reach id (dimension 2)
- __Discharge point information (csv)__
This is a file to define the coordinates (lat/lon) and upstream area of each river id in your river discharge data.
  * The sample format is:  
  
|id|lat|lon|uparea|  
|---|---|---|---|  
|0|35.11|-120.24|300.25|  
  
You can find the sample dataset in an example/discharge/ directory.  

In [3]:
dschgFile = "./examples/discharge/discharge_sample.nc"
pointInfoFile = "./examples/discharge/pointInfoFile_sample.csv"

Here are sample files' description.  

In [4]:
import xarray as xr
xr.open_dataset(dschgFile)

<xarray.Dataset>
Dimensions:    (id: 28998, time: 5)
Coordinates:
  * id         (id) int64 74042215 74042216 74044411 ... 74013196 74013320
  * time       (time) datetime64[ns] 1984-01-01 1984-01-02 ... 1984-01-05
Data variables:
    Discharge  (time, id) float32 ...

In [8]:
import pandas as pd
df = pd.read_csv(pointInfoFile)
df.head()

,id,lat,lon,uparea
0,74042215,38.811666,-90.117499,1.343325e+06
1,74042216,38.841666,-90.233333,1.343175e+06
2,74044411,38.841666,-90.233333,1.155583e+02
3,74042217,38.821666,-90.388333,1.343040e+06
4,74044533,38.821666,-90.388333,4.826931e+01


Note that those lat/lon in this pointInfoFile are the locations for your discharge data, which might be the result from other model (that is, other model grid coordinates).  

lifpy.Forcing.makeForcing() try to adjust those original (your discharge data coordinates) to the LISFLOOD-FP model coordinates (that is, lat/lon coordinates what you created from above hydrography) by matching the upstream area size within a reasonable buffer.  

In [14]:
lfp.Forcing.makeForcing(dschgFile, pointInfoFile, ex="example")

./out/setting_example.bci
./out/inflow_example.bdy


You can find the output in out/ directory:  

In [16]:
!head -n5 ./out/setting_example.bci

P -90.11916666666667 38.81333333333333 QVAR inflow_74042215
P -90.23083333333334 38.840833333333336 QVAR inflow_74042216
P -90.23416666666667 38.840833333333336 QVAR inflow_74044411
P -90.385 38.821666666666665 QVAR inflow_74042217
P -90.38916666666667 38.82083333333333 QVAR inflow_74044533


In [18]:
!head -n6 ./out/inflow_example.bdy

inflow [m3/s] from 1984-01-01T00:00:00.000000000 to 1984-01-05T00:00:00.000000000
inflow_74042215
5 seconds
3963.076904296875 0
3799.87548828125 86400
3633.43408203125 172800


Note that lifpy reads the datetime index information from your netCDF discharge data, and make 2nd column (seconds) dynamically.  

Done! Now you have all data to run the subgrid version of LISFLOOD-FP.  
Run the simulation by:
```bash
${executable} ${your .par file}
```

# Visualizing results  
Current lifpy only supports the instant visualization of a snapshot (at specific time) of output file (e.g. res-001.txt). Please see the Todo in the document to see the future implementation.

lifpy has a higher API for instant visualization to check your simulation:
```python
fileName = "your results (.txt) from LISFLOOD-FP"
name = "name of the result (e.g. width, elevation, etc.)"
cacheFile = "path to the cached netCDF file that lifpy.PreProcess.mfpreprocess generates."
# in default it is in cache/uparea.nc
img = lifpy.Visualize.show(fileName, name, cacheFile)```

The sample result is stored in examples/result/ directory:  

In [23]:
fileName = "./examples/result/res_sample.wd"
name = "RiverWidth"
cacheFile = "./cache/uparea.nc"
img = lfp.Visualize.show(fileName, name, cacheFile)
img

ValueError: Invalid file path or buffer object type: <class 'src.visualize.Visualize'>

This API uses a datashader as a pipeline, and the grids are dynamically regridded which enables smoother loading.  
This is especially useful when you see a big picture of your simulation (which has more than a millions of data points in a large scale hi-res modeling) which would otherwize be very expensive to visualize.  